# Security Dataset Generation
This Notebook is intended to generate a security dataset. So far it's columns will consist of paragraph scenarios intended to feed into the LLM we will fine tune

### Prompt Template
Prompt: Generate a security scenario involving a buisness. The scenario should include specific risks or vulnerabilities, such as cloud systems, employee actions, or third-party services. The scenario should also allud to potential security attacks that could happen in the organization.

### OpenAI

In [ ]:
attack_map = {
    "Phishing": "A social engineering attack to steal sensitive information by impersonating a trustworthy entity.",
    "Ransomware": "Malware that encrypts a victim's files, demanding a ransom to restore access.",
    "Man-in-the-Middle": "An attack where a hacker intercepts communication between two parties to steal data.",
    "Insider Threats": "Security risks originating from within the organization, such as malicious employees.",
    "DDoS": "Distributed denial-of-service attacks to overwhelm systems, causing disruption of services.",
    "SQL-injection": "Exploiting a web app's database query to access or manipulate data",
    "Cross-Site-Scripting": "Injecting malicious scripts into web pages viewed by others.",
    "Privilege Escalation": "This attack exploits vulnerabilities within business systems to gain higher access levels and steal or manipulate data."
}

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import openai
import random
import json
import os

# Set up your OpenAI API key
openai.api_key = os.getenv('OPENAI_API_KEY')

# Function to generate a scenario from GPT-4
def generate_scenario():
    prompt = """
    Generate a security scenario involving a buisness. The scenario should include specific risks or vulnerabilities, 
    such as cloud systems, employee actions, or third-party services. The scenario should also allud to potential security 
    attacks that could happen in the organization."""
    response = openai.completions.create(
        model= "gpt-3.5-turbo", 
        prompt=prompt,
        max_tokens=150,  # Adjust based on desired scenario length
        temperature=0.7,  # Adjust creativity
    )
    return response['choices'][0]['text'].strip()

# Main function to generate 1,000 scenarios
def generate_dataset(num_scenarios=1000):
    dataset = []
    
    for i in range(num_scenarios):
        scenario = generate_scenario()
        # attacks = map_attacks_to_scenario(scenario)
        
        entry = {
            "id": i + 1,
            "scenario": scenario,
            # "attacks": [attack_map[attack] for attack in attacks]
        }
        dataset.append(entry)
        
        # Print progress
        if (i + 1) % 100 == 0:
            print(f"Generated {i + 1} scenarios.")
    
    return dataset

In [ ]:
models = openai.models.list()
models

In [ ]:
df = generate_dataset()

# # Optionally, save to a JSON or CSV file
# with open('generated_dataset.csv', 'w') as f:
#     json.dump(df, f, indent=4)

#  Save as CSV
df.to_csv("output.csv", index=False)

print("Dataset generated and saved!")

### HuggingFace

In [ ]:
# !pip install transformers datasets torch

In [34]:
# Example list of prompts (business context descriptions without attack types)
ex_prompts = [
    "A healthcare organization stores patient records in a cloud-based system. Doctors and nurses access these records remotely using both work and personal devices. The organization also relies on third-party APIs to facilitate insurance claims processing. Recently, some employees have received suspicious emails requesting login credentials. Additionally, the hospital's IoT medical devices are connected to the same network as administrative systems.",
    "An e-commerce company stores sensitive customer data such as credit card information, order history, and delivery addresses. Employees have access to the database, but some are concerned about data breaches. Recently, several users reported strange transactions in their accounts that they did not initiate. The company's website also relies on several third-party payment gateways.",
    "A university's online system allows students to access grades, course materials, and financial information. Teachers and administrators update grades and assign courses. Recently, some students reported unauthorized access to their grades by unknown users. The system is connected to several external platforms for online learning and exams.",
    "A financial institution handles large transactions and sensitive client data, including investment portfolios. Employees access client records remotely, and the institution has implemented multi-factor authentication. Despite these measures, recent suspicious login attempts have occurred, especially from overseas. The bank also uses third-party services for processing transactions and investments.",
    "A manufacturing company uses IoT sensors to monitor equipment and ensure production efficiency. The factory's network is connected to the internet, and employees use remote access to check status reports. Recently, some employees noticed unusual activity in the system, with sensors transmitting abnormal data. The company also uses cloud-based services for inventory management."
]

In [ ]:
import torch
import csv
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load pre-trained model and tokenizer
model_name = 'gpt2' 
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Set the pad_token to eos_token (common practice with GPT2 models)
tokenizer.pad_token = tokenizer.eos_token

# Ensure the model is in evaluation mode
model.eval()

# Function to generate a scenario based on a prompt
def generate_scenario(prompt, max_length=150):
     # Tokenize the prompt with padding and return the attention mask
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=512)

    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']  # Extract the attention mask

     # Use the attention mask during model generation
    with torch.no_grad():
        outputs = model.generate(input_ids, attention_mask=attention_mask, max_length=max_length, num_return_sequences=1, no_repeat_ngram_size=2, temperature=0.7)
    
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Generate scenarios using multiple prompts
generated_scenarios = []
num_scenarios = 10
for i in range(num_scenarios):  # Change this number for more or fewer scenarios
    prompt = random.choice(ex_prompts)  # Randomly select a prompt from the list
    generated_scenario = generate_scenario(prompt)
    
    # Store the generated scenario
    generated_scenarios.append({
        'scenario': generated_scenario
    })

# Save the scenarios as a CSV file
with open("generated_scenarios.csv", mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=['scenario'])
    writer.writeheader()
    
    for entry in generated_scenarios:
        writer.writerow(entry)

print(f"Dataset generation complete. {num_scenarios} scenarios with attack types saved.")


IndexError: too many indices for tensor of dimension 2